In [ ]:
import inspect
from pprint import pprint
import torch
from torch_geometric.utils import scatter_




class LinkMessagePassing(torch.nn.Module):
    r"""Base class for creating message passing layers

    .. math::
        \mathbf{x}_i^{\prime} = \gamma_{\mathbf{\Theta}} \left( \mathbf{x}_i,
        \square_{j \in \mathcal{N}(i)} \, \phi_{\mathbf{\Theta}}
        \left(\mathbf{x}_i, \mathbf{x}_j,\mathbf{e}_{i,j}\right) \right),

    where :math:`\square` denotes a differentiable, permutation invariant
    function, *e.g.*, sum, mean or max, and :math:`\gamma_{\mathbf{\Theta}}`
    and :math:`\phi_{\mathbf{\Theta}}` denote differentiable functions such as
    MLPs.
    See `here <https://rusty1s.github.io/pytorch_geometric/build/html/notes/
    create_gnn.html>`__ for the accompanying tutorial.

    """

    def __init__(self, aggr='add'):
        super(LinkMessagePassing, self).__init__()
        
        # this calls will get all params of the self.message() func
        # defined in the instance that inherits from this class
        #  ecample def message(self, x_j, norm) -> x_j and norm
        self.message_args = inspect.getargspec(self.message)[0][1:]
        # with GCN this is [x_j, norm]
        self.update_args = inspect.getargspec(self.update)[0][2:]
        # with GCN this is []

    #def propagate(self, aggr, edge_index, edge_neighbors, **kwargs):
    #def propagate(self, aggr, edge_index, **kwargs):
    def propagate(self, aggr, edge_neighbors, **kwargs):
        r"""The initial call to start propagating messages.
        Takes in an aggregation scheme (:obj:`"add"`, :obj:`"mean"` or
        :obj:`"max"`), the edge indices, and all additional data which is
        needed to construct messages and to update node embeddings."""

        assert aggr in ['add', 'mean', 'max']
        #kwargs['edge_index'] = edge_index
        kwargs['edge_neighbors'] = edge_neighbors
        #print("self update_args: ", self.update_args)
        
        #print("self message_args: ",self.message_args)
        #print("kwargs: ",kwargs)
        size = None
        message_args = []
        for arg in self.message_args:
            if arg[-2:] == '_i':
                tmp = kwargs[arg[:-2]]
                size = tmp.size(0)
                message_args.append(tmp[edge_index[0]])
            elif arg[-2:] == '_j':
                tmp = kwargs[arg[:-2]]
                size = tmp.size(0)
                message_args.append(tmp[edge_index[1]]) #appends x tensors of the selected nodes in edge list
            elif arg[-2:] == '_l':
                # this branch is for link/edge feature based messages
                """
                   create a edge_neighborhood list
                   this lists says which edge index corresponds to neighbors of the current edge
                   the messages passed will correspond to those of the edge_neighbors
                  
                   some idea would be: use the same, just change messages from links and change
                   x features to correspond to edges, maybe add as xedge
                   
                   so message would be xedges[neighbor_edge_list]
                   
                   norm would be the same           
                   
                   way propagate is called changes from LinkGCNConv
                   message and update parameteres change also
                   
                   maybe: the aggregate could take into account that there's 2 ends of an edge,
                   like aggregating from each side and doing a mean 

                """
                tmp = kwargs[arg[:-2]] # this is the values of the edge betweennesses for each edge
                size = tmp.size(0)
                message_args.append(tmp[edge_neighbors[0]]) #appends x tensors of the selected nodes in edge list
            else:
                message_args.append(kwargs[arg]) # appends full norm tensor
        #print("message_args: ")
        #pprint(message_args)
        update_args = [kwargs[arg] for arg in self.update_args]
        
        #print("update_args: ", update_args)
        out = self.message(*message_args)

        #print("\n normalized messages (edge attribute) ")
        #pprint(out)
        #print("\n edge_neighbor index used ")
        #pprint(edge_neighbors[0])

        
        out = scatter_(aggr, out, edge_neighbors[0], dim_size=size)
        #print("\n scatter output ")
        #pprint(out)

        out = self.update(out, *update_args)

        #print("\n")

        return out

    def message(self, x_j):  # pragma: no cover
        r"""Constructs messages in analogy to :math:`\phi_{\mathbf{\Theta}}`
        for each edge in :math:`(i,j) \in \mathcal{E}`.
        Can take any argument which was initially passed to :meth:`propagate`.
        In addition, features can be lifted to the source node :math:`i` and
        target node :math:`j` by appending :obj:`_i` or :obj:`_j` to the
        variable name, *.e.g.* :obj:`x_i` and :obj:`x_j`."""

        return x_j

    def update(self, aggr_out):  # pragma: no cover
        r"""Updates node embeddings in analogy to
        :math:`\gamma_{\mathbf{\Theta}}` for each node
        :math:`i \in \mathcal{V}`.
        Takes in the output of aggregation as first argument and any argument
        which was initially passed to :meth:`propagate`."""

        return aggr_out


In [ ]:
import torch
from torch.nn import Parameter
from torch_scatter import scatter_add
#from torch_geometric.nn.conv import MessagePassing
#from torch_geometric.nn.conv import LinkMessagePassing
from torch_geometric.utils import add_self_loops

from torch_geometric.nn.inits import glorot, zeros
#from ..inits import glorot, zeros


class LinkGCNConv(LinkMessagePassing):
    r"""The graph convolutional operator from the `"Semi-supervised
    Classfication with Graph Convolutional Networks"
    <https://arxiv.org/abs/1609.02907>`_ paper

    .. math::
        \mathbf{X}^{\prime} = \mathbf{\hat{D}}^{-1/2} \mathbf{\hat{A}}
        \mathbf{\hat{D}}^{-1/2} \mathbf{X} \mathbf{\Theta},

    where :math:`\mathbf{\hat{A}} = \mathbf{A} + \mathbf{I}` denotes the
    adjacency matrix with inserted self-loops and
    :math:`\hat{D}_{ii} = \sum_{j=0} \hat{A}_{ij}` its diagonal degree matrix.

    Args:
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        improved (bool, optional): If set to :obj:`True`, the layer computes
            :math:`\mathbf{\hat{A}}` as :math:`\mathbf{A} + 2\mathbf{I}`.
            (default: :obj:`False`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
    """

    def __init__(self, in_channels, out_channels, improved=False, bias=True):
        super(LinkGCNConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)

    def forward(self, x, edge_neighbors, edge_weight=None):
        """
            needs to be changed to have:
            - edge_neighborhoods
            - xedge as features for edges
            - aggregate function 

        """
        if edge_weight is None:
            edge_weight = torch.ones(
                (edge_neighbors.size(1), ), dtype=x.dtype, device=x.device)
        edge_weight = edge_weight.view(-1)
        assert edge_weight.size(0) == edge_neighbors.size(1)

        edge_neighbors = add_self_loops(edge_neighbors, num_nodes=x.size(0))
        loop_weight = torch.full(
            (x.size(0), ),
            1 if not self.improved else 2,
            dtype=x.dtype,
            device=x.device)
        edge_weight = torch.cat([edge_weight, loop_weight], dim=0)

        row, col = edge_neighbors
        deg = scatter_add(edge_weight, row, dim=0, dim_size=x.size(0))
        deg_inv = deg.pow(-0.5)
        deg_inv[deg_inv == float('inf')] = 0

        norm = deg_inv[row] * edge_weight * deg_inv[col]

        x = torch.matmul(x, self.weight)
        return self.propagate('add', edge_neighbors, x=x, norm=norm)

    def message(self, x_l, norm):
        return norm.view(-1, 1) * x_l

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)


In [28]:
import torch
import torch.nn.functional as F
#from torch_geometric.nn import GCNConv
#from torch_geometric.nn import LinkGCNConv
import torch.nn as nn
from pprint import pprint

import networkx as nx
import time
from torch_geometric.data import DataLoader
import importlib
from torch_geometric.data import Data
import pickle
import numpy as np








class MyOwnDataset2():
    def __init__(self,  root, name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        #print(self.data)
        #print(self.data.edge_index)
        #print(self.data.num_features)
        self.num_features = self.data.num_features
        self.num_classes = 1
        self.filename = name
        
        # prepare edge_neighbors
        edges_dict = {}
        i=0
        for edge in self.data.edge_index[0]:
            edges_dict[i]= (self.data.edge_index[0][i],
                            self.data.edge_index[1][i])
            i+=1
            
        #print("\n edges_dict:")
        #pprint(edges_dict)
        #print("\n")
        """
            {
            0 : (1,3),
            1 : (1,4),
            2 : (2,0)
            ...
            }
            
            edge_neighbors= [[],[]]
            for edge in edges_dict.keys():
                for node in edges_dict[edge]:
                    for edge2 in edges_dict.keys():
                        if edge2 != edge and \
                           ( edges_dict[edge2][0] == node or \
                             edges_dict[edge2][1] == node ):
                             edge_neighbors[0].append(edge)
                             edge_neighbors[1].append(edge2)
                             
        """
        edge_neighbors= [[],[]]
        for edge in edges_dict.keys():
            for node in edges_dict[edge]:
                for edge2 in edges_dict.keys():
                    if edge2 != edge and ( edges_dict[edge2][0] == node or edges_dict[edge2][1] == node ):
                        edge_neighbors[0].append(edge)
                        edge_neighbors[1].append(edge2)
        
        self.data.edge_neighbors = torch.LongTensor(edge_neighbors)
        
        #print()
        #print("edge_neighbors")
        #pprint(self.data.edge_neighbors)
        #print()
        #print(type(self.data.edge_neighbors))
        #print()
        
        f.close()

class NetLGCN1(torch.nn.Module):
    def __init__(self, d1=16):
        super(NetLGCN1, self).__init__()
        self.conv1 = LinkGCNConv(1, d1)
        self.conv2 = LinkGCNConv(d1, 1)
        self.d1 = d1
        

    def forward(self, data):
        x, edge_neighbors = data.x, data.edge_neighbors

        x = self.conv1(x, edge_neighbors)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_neighbors)

        # output as multiclass target
        #return F.log_softmax(x, dim=1)
        
        # output as regression target
        return x
    
    def __str__(self):
        return "Net1-gcn(%d,%d)-gcn(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                               dataset.num_classes)

class NetLGCN2(torch.nn.Module):
    def __init__(self, d1=16,d2=16):
        super(NetLGCN2, self).__init__()
        self.conv1 = LinkGCNConv(dataset.num_features, d1)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, dataset.num_features)
        self.d1 = d1
        self.d2 = d2
        

    def forward(self, data):
        x, edge_neighbors = data.x, data.edge_neighbors

        x = self.conv1(x, edge_neighbors)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as multiclass target
        #return F.log_softmax(x, dim=1)
        
        # output as regression target
        return x
    
    def __str__(self):
        return "Net1-gcn(%d,%d)-gcn(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                               dataset.num_classes)
    
    
class Net2(torch.nn.Module):
    def __init__(self, d1=300,d2=100):
        super(Net2, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, d1 )
        #self.conv2 = GCNConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, dataset.num_features)
        self.d1 = d1
        self.d2 = d2

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    
    def __str__(self):
        return "Net2-gcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                                        self.d2,self.d2,
                                                        dataset.num_features)

    
    
class Net3(torch.nn.Module):
    def __init__(self, d1=300,d2=100):
        super(Net3, self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, d1 )
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, dataset.num_features)
        self.d1 = d1
        self.d2 = d2

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    def __str__(self):
        return "Net3-SAGEgcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                                        self.d2,self.d2,
                                                        dataset.num_features)

    


def loadDataset(collection, name=None):
    try:
        # import datasets
        themodule = importlib.import_module("torch_geometric.datasets")
        # get the function corresponding to collection
        method_to_call = getattr(themodule, collection)
        if name:
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
    except:
        # custom module
        method_to_call = globals()[collection]
       
        if name:
            
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
        


def transformMask(mask):
    train_mask = []
    i = 0
    for pick in mask:
        if pick[0]==1:
            train_mask.append(i)
        i+=1
    return train_mask


def shuffleTrainTestMasks(data, trainpct = 0.7):
    ysize = list(data.y.size())[0]
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[int(ysize*trainpct):] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
  

def shuffleTrainTestValMasks(data, trainpct = 0.7, valpct = 0.2):

    ysize = list(data.y.size())[0]
    #print("total ", ysize)
    #print(" train ",int(ysize*trainpct)-int(ysize*trainpct*valpct))
    #print(" val ",int(ysize*trainpct*valpct))
    #print(" test ",int(ysize*(1- trainpct) ))
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[:int(ysize*trainpct)] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    #print(" train sum ",data.train_mask.sum())
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    #print(" test sum ",data.test_mask.sum())
    
    # transform to list of indexes
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
    
    data.val_mask = data.train_mask[:int(ysize*trainpct*valpct)]
    data.train_mask = data.train_mask[int(ysize*trainpct*valpct):]

    
    #print(data.train_mask)
    #print(data.val_mask)
    #print(data.test_mask)
    
    

def trainTestEval(dataset, epochs=1, batch_size=32):
    global Net
    loader = DataLoader(dataset,  shuffle=False)
    i = 0
    #print(loader)
    #print(dir(loader))
    
    G = dataset.data
    print(G)
    start = time.time()


    # 1.  prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #print("using ",device)
    model = Net.to(device)  
    data = G.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    # 2.  create a train_mask, and a test_mask (val_mask for further experiments)
    #shuffleTrainTestMasks(data)
    #shuffleTrainTestValMasks(data)
    shuffleTrainTestMasks(data)

    # 3. train some epochs
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 25 == 0 :
            print("epoch-loss: ",epoch, loss)

    # 4. Model evaluation
    model.eval()
    #  classification in a multiclass setting
    #_, pred = model(data).max(dim=1)
    #correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    #acc = correct / data.test_mask.sum().item()
    #print('Accuracy: {:.4f}'.format(acc))


    # regression 
    pred = model(data)
    #print("target: ",data.y[data.test_mask])
    #print("prediction: ",pred[data.test_mask])
    #print(pred[data.test_mask].type())
    #print(data.y[data.test_mask].type())
    
    # prepare the normalized mean root squared error
    t = data.y[data.test_mask]
    y = pred[data.test_mask]
    nrmse = torch.sum((t - y) ** 2)/len(data.test_mask)
    nrmse = nrmse.sqrt()
    print("RMSE: ",nrmse)

    #m = torch.mean(t)
    #print("mean",m)
    #tmax = torch.max(t)
    #tmin = torch.min(t)
    #sd = tmax-tmin
    #print("sd",sd)
    #nrmse = (nrmse - m)/sd
    #print("NRMSE:",nrmse)


    endtime = time.time()
    print("Total train-test time: "+str(endtime-start))
    
    with open("results.txt","a") as f:
        #print(dir(dataset))
        f.write("\n")
        f.write(str(model)+" " 
                +str(dataset.filename)+" "  
                +"nrmse: "+str(nrmse.item())+" " 
                +"total time: "+str(endtime-start) 
                +" negative vals?: "+str(False) 
                +"\n"
               )
    
    del model

    #i+=1
    #if i==1:
    #    break

In [2]:
#dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_10_0_10_nb.pickle')
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/er_5_0_45_eb.pickle')
#print(dir(dataset.data))
#print()
global Net
Net=Net1(d1=5)
trainTestEval(dataset,  epochs=1)
del Net

Data(edge_index=[2, 7], x=[7, 1], y=[7])
tensor([[0, 0, 1, 1, 1, 2, 3],
        [1, 2, 2, 3, 4, 4, 4]])
Data(edge_index=[2, 7], edge_neighbors=[2, 28], x=[7, 1], y=[7])
epoch-loss:  0 tensor(0.9520, device='cuda:0', grad_fn=<MseLossBackward>)
RMSE:  tensor(0.9339, device='cuda:0', grad_fn=<SqrtBackward>)
Total train-test time: 1.9290800094604492


In [16]:
#dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_10_0_10_nb.pickle')
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/er_5_0_45_eb.pickle')
#print(dir(dataset.data))
#print()
global Net
Net=Net1(d1=55)
trainTestEval(dataset,  epochs=200)
del Net

Data(edge_index=[2, 7], edge_neighbors=[2, 28], x=[7, 1], y=[7])
epoch-loss:  0 tensor(0.3592, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(0.0284, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(0.0017, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(0.0013, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor(0.0014, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  150 tensor(0.0008, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  175 tensor(0.0010, device='cuda:0', grad_fn=<MseLossBackward>)
RMSE:  tensor(0.1227, device='cuda:0', grad_fn=<SqrtBackward>)
Total train-test time: 0.541536808013916


In [17]:
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/er_5_0_45_eb.pickle')
global Net
Net=Net1(d1=55)
trainTestEval(dataset,  epochs=200)
del Net

# RMSE is between 15% adn 8% in a transductive setting

Data(edge_index=[2, 7], edge_neighbors=[2, 28], x=[7, 1], y=[7])
epoch-loss:  0 tensor(0.0720, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(0.0050, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(0.0048, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(0.0034, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(0.0033, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor(0.0023, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  150 tensor(0.0024, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  175 tensor(0.0024, device='cuda:0', grad_fn=<MseLossBackward>)
RMSE:  tensor(0.0835, device='cuda:0', grad_fn=<SqrtBackward>)
Total train-test time: 0.5560200214385986


In [33]:
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/TUDataset_PROTEINS_996_eb.pickle')
global Net
Net=NetLGCN2(d1=250,d2=50)
trainTestEval(dataset,  epochs=400)
del Net

Data(edge_index=[2, 16], edge_neighbors=[2, 88], x=[16, 1], y=[16])
epoch-loss:  0 tensor(0.0204, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  150 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  175 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  200 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  225 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  250 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  275 tensor(0.0168, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  300 tensor(0